In [1]:
from quchem_ibm.Qiskit_Chemistry import *

In [2]:
transformation='BK'

In [3]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start

Molecule = 'LiH'
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

(-4.142299396835104+0j) [] +
(0.029712582409311572+0j) [X0 X1 X2] +
(-0.001577981883357554+0j) [X0 X1 X2 X3 Y7 Y11] +
(0.0009529034207233027+0j) [X0 X1 X2 Y3 Y5] +
(0.0019619556294531754+0j) [X0 X1 X2 Z3] +
(0.002934941328246141+0j) [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
(0.0034056263156498285+0j) [X0 X1 Z2 Y3 Y4 X5] +
(-0.0011759180512873526+0j) [X0 X1 X3 X4 Y7 Y11] +
(0.0011649814581185156+0j) [X0 X1 X3 Y4 Y5 Z6 Z7] +
(0.0011759180512873526+0j) [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
(0.002550980665913645+0j) [X0 X1 X3 Z4 Y5 Y6 Z7] +
(-0.002316892946412113+0j) [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
(-0.0011409748951247602+0j) [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
(0.0015099541958032813+0j) [X0 X1 X3 X6 Y7 Y11] +
(-0.0015033808265301452+0j) [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
(0.0015099541958032811+0j) [X0 X1 X3 Y7 X8 Y11] +
(-0.0015099541958032811+0j) [X0 X1 X3 Y7 Y8 Z10 X11] +
(-0.0015033808265301487+0j) [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
(6.573369273132568e-06+0j) [X0 X1 X3 Y7 Z8 Y10 X11] +
(-0.0007014172134443986+0j) [X0

From PHYS. REV. X, **8**, 031022 (2018):

$$LiH_{BK}^{HartreeFock} = | 101000000000\rangle$$

In [4]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

JW ground state =  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
BK ground state =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
# qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
# print('qubits only acted on by I or Z:', qubits_to_remove)


# input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# # Remove_Z_terms_from_Hamiltonian
# NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
#                                                     QubitHamiltonian, 
#                                                     input_state,
#                                                     qubits_to_remove,
#                                                     check_reduction=True)
# NewQubitHamiltonian

In [6]:
# qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
# NewQubitHamiltonian_relabelled

In [5]:
NewQubitHamiltonian_relabelled=QubitHamiltonian

# Find what new FCI energy is
- here should be the same as true answer!

In [6]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

# new_Molecular_H_MATRIX = np.flip(new_Molecular_H_MATRIX)

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

new_FCI =  (-7.97118431556568-1.1849528074038059e-16j) VS old FCI: -7.971184315565542
True


In [7]:
np.dot(ground_state_vector.conj().T, new_Molecular_H_MATRIX.dot(ground_state_vector))

(-7.971184315565686-3.7947076036992655e-19j)

# HF + UCCSD

In [10]:
# input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# new_input_state = np.delete(input_state, qubits_to_remove)

# n_qubits_new = len(new_input_state)
# new_input_state

In [8]:
new_input_state=ansatz_obj.Get_BK_HF_state_in_OCC_basis()

In [12]:
# ansatz_obj.Get_ia_and_ijab_terms()

# print('ia standard')
# print(ansatz_obj.Sec_Quant_CC_ia_Fermi_ops)

# print('ijab standard')
# print(ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

# ### single trotter step
# ansatz_obj.UCCSD_single_trotter_step(transformation,
#                                      List_FermiOps_ia=ansatz_obj.Sec_Quant_CC_ia_Fermi_ops,
#                                      List_FermiOps_ijab=ansatz_obj.Sec_Quant_CC_ijab_Fermi_ops)

# print('')
# print('ia standard')
# print(ansatz_obj.Second_Quant_CC_single_Trot_list_ia)
# print('ijab standard')
# print(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab)

In [13]:
# new_CC_ia_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ia,
#                                                       qubits_to_remove)
# print('ia reduced')
# print(new_CC_ia_single_trot)

# new_CC_ijab_single_trot = Remove_indices_from_qubit_ops(ansatz_obj.Second_Quant_CC_single_Trot_list_ijab,
#                                                       qubits_to_remove)
# print('ijab reduced')
# print(new_CC_ijab_single_trot)


# relabelled_new_CC_ia_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ia_single_trot)
# relabelled_new_CC_ijab_single_trot =Re_label_qubit_operators(qubitNo_re_label_dict, new_CC_ijab_single_trot)

# print('')
# print('ia reduced relabelled')
# print(relabelled_new_CC_ia_single_trot)
# print('')
# print('ijab reduced relabelled')
# print(relabelled_new_CC_ijab_single_trot)

# Ansatz Circuit

In [14]:
# for op in relabelled_new_CC_ia_single_trot[1]:
#     print(op)
# print(list(op.terms.keys())[0])

In [15]:
# UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(new_input_state,
#                                      relabelled_new_CC_ia_single_trot, 
#                                      relabelled_new_CC_ijab_single_trot)

# theta_ia = [0 for _ in range(len(relabelled_new_CC_ia_single_trot))]
# theta_ijab = [0 for _ in range(len(relabelled_new_CC_ijab_single_trot))]

# UCCSD_ansatz_Q_Circ =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
#                                         Theta_param_list_ia=theta_ia, 
#                                          Theta_param_list_ijab=theta_ijab,
#                                          ia_first=True)
# UCCSD_ansatz_Q_Circ

In [10]:
n_qubits= len(new_input_state)
check_ansatz_state = False

Ansatz_circuit, q_reg = Vector_defined_Ansatz(n_qubits, ground_state_vector, check_ansatz_state=check_ansatz_state)

In [11]:
standard_VQE_circuits, standard_I_term = Build_Standard_VQE_circuits(
                                                    NewQubitHamiltonian_relabelled, 
                                                    Ansatz_circuit,
                                                    q_reg)

In [13]:
list(standard_VQE_circuits)

Getting_standard_VQE_circuits: 63it [03:32,  2.37s/it]

KeyboardInterrupt: 

# Graph

In [17]:
from tqdm.notebook import tqdm

In [18]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets

Building Graph Edges: 100%|##########| 631/631 [00:16<00:00, 38.86it/s] 


{0: [(-4.142299396835097+0j) []],
 1: [(0.07828114149315567+0j) [Z9]],
 2: [(0.07828114149315564+0j) [Z3 Z5 Z7]],
 3: [(-0.22752612049271337+0j) [Z8],
  (0.004218766523001721+0j) [Z3 Z5 Y6 Z7 Y8],
  (0.004878950745532889+0j) [Z1 Y2 X3 Y7 X8 Z10 X11],
  (0.003557129611273106+0j) [Z3 Y4 X5 Y7 X8 Z10 X11],
  (0.004912839124340319+0j) [X8 Z9 X10],
  (0.001509954195803291+0j) [Y0 X1 X3 Y7 X8 Z10 X11]],
 4: [(-0.22752612049271337+0j) [Z8 Z9],
  (0.004218766523001721+0j) [Y6 Y8 Z9],
  (0.002550980665913573+0j) [Z0 X1 Y3 Y4 X5 X8 Z9],
  (-0.001509954195803291+0j) [Z0 X1 X3 Y7 X8 Y10 X11],
  (0.0018744069004631339+0j) [Z0 Y1 Y2 X8 Z9],
  (0.0025149236902432663+0j) [X0 X8 Z9]],
 5: [(-0.2275261204927134+0j) [Z6],
  (0.004218766523001721+0j) [Z3 Z5 X6 Z7 X8],
  (-0.0015099541958032913+0j) [X0 X1 Y3 Z5 Y6 X7 Z9 Z10 X11],
  (0.0025149236902432663+0j) [Y0 Z3 Z5 Y6 Z7],
  (0.0018744069004631332+0j) [X0 Y1 Z2 Z3 Z5 Y6 Z7],
  (0.0025509806659135734+0j) [X0 X1 X3 Z4 Y5 Y6 Z7]],
 6: [(-0.2275261204927134

# Seq Rot circuits

In [20]:
n_qubits= len(new_input_state)
rotation_reduction_check=False

Seq_Rot_VQE_circuits, Seq_Rot_I_term = Get_Seq_Rot_Unitary_Part_circuits(
                                anti_commuting_sets, 
                                Ansatz_circuit,
                                q_reg,
                                n_qubits,
                                S_index_dict=None, 
                                rotation_reduction_check=rotation_reduction_check)

# LCU circuits

In [ ]:
n_qubits= len(new_input_state)
check_ansatz_state = False

LCU_VQE_circuits, LCU_I_term= Get_LCU_Unitary_Part_circuits(anti_commuting_sets, ground_state_vector, n_qubits, 
                                  N_index_dict=None,check_ansatz_state=check_ansatz_state)

# Save experiment input

In [ ]:
list(NewQubitHamiltonian_relabelled)

In [ ]:
filename = 'LiH_bravyi_kitaev_12_qubit_experiment'
n_qubits= len(new_input_state)

Save_exp_inputs(filename, NewQubitHamiltonian_relabelled, anti_commuting_sets, Hamilt.geometry, basis, transformation,
                    Graph_colouring_strategy, fci_energy,
                    standard_VQE_circuits, standard_I_term,
                    Seq_Rot_VQE_circuits, Seq_Rot_I_term,
                    LCU_VQE_circuits, LCU_I_term,
                    ground_state_vector,
                    n_qubits,
                    S_index_dict=None,
                    N_index_dict=None)